<a href="https://colab.research.google.com/github/inshra12/ESM-MLP-baseline-protein-function/blob/main/notebooks/sequence-based-function-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 1

## Importing files

In [88]:
from data_loader import load_data,encode_labels

In [89]:
!pip install fair-esm

In [90]:
from esm_embedder import save_embeddings

## Loading Data

In [91]:
df=load_data("/content/Protein_data(265).csv")

First protein entry:
 ID                                                     O00222
Sequence    MVCEGKRSASCPCFFLLTAKFYWILTMMQRTHSQEYAHSIRVDGDI...
BP          GO:0007196; GO:0007193; GO:0007216; GO:0051966...
CC                                                 GO:0005886
MF                         GO:0004930; GO:0008066; GO:0001642
Labels      [ GO:0007601,  GO:0008066,  GO:0051966, GO:000...
Name: 0, dtype: object


In [92]:
#save_embeddings("/content/Protein_data(265).csv", "cls_embeddings.csv")

In [93]:
import pandas as pd
# Load ESM embeddings (previously extracted)
esm_df = pd.read_csv("/content/cls_embeddings.csv")  # must have column 'ID'

## Encoding multi-hot labels

In [94]:
encoded_labels, mlb = encode_labels(df["Labels"])

Number of unique GO terms: 1995


In [95]:
# Step 2: Convert encoded labels into a DataFrame
encoded_labels_df = pd.DataFrame(encoded_labels, columns=mlb.classes_)
encoded_labels_df["ID"] = df["ID"]  # Add the ID column
# Move "ID" column to the front
cols = list(encoded_labels_df.columns)
cols.remove("ID")
new_order = ["ID"] + cols
final_df = encoded_labels_df[new_order]
final_df = final_df.loc[:, final_df.columns != '']
final_df

,ID,GO:0000122,GO:0000139,GO:0000165,GO:0000226,GO:0000278,GO:0000578,GO:0001501,GO:0001503,GO:0001516,GO:0001525,GO:0001530,GO:0001540,GO:0001558,GO:0001569,GO:0001570,GO:0001586,GO:0001587,GO:0001588,GO:0001591,GO:0001594,GO:0001601,GO:0001602,GO:0001604,GO:0001605,GO:0001607,GO:0001609,GO:0001614,GO:0001616,GO:0001618,GO:0001621,GO:0001626,GO:0001634,GO:0001635,GO:0001639,GO:0001640,GO:0001641,GO:0001642,GO:0001649,GO:0001650,...,GO:0051393,GO:0060055,GO:0060070,GO:0060079,GO:0060170,GO:0061844,GO:0070098,GO:0070161,GO:0070509,GO:0070553,GO:0070915,GO:0071222,GO:0071257,GO:0071380,GO:0071385,GO:0071392,GO:0071415,GO:0071420,GO:0071482,GO:0071875,GO:0071880,GO:0071886,GO:0071944,GO:0072562,GO:0090534,GO:0095500,GO:0097242,GO:0097449,GO:0097731,GO:0097746,GO:0098978,GO:0098981,GO:1901363,GO:1902093,GO:1903143,GO:1903413,GO:1904646,GO:1990409,GO:1990430,GO:1990763
0,O00222,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,O14842,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,O15303,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,O43603,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,O43613,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Q9Y271,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
261,Q9Y2T5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
262,Q9Y2T6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
263,Q9Y5N1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Merge sequence embedding and labels in one file

In [96]:
# Drop the ID column (non-binary) and sum over the rows
go_term_counts = final_df.drop(columns=["ID"]).sum(axis=0).sort_values(ascending=False)
print(go_term_counts)  # Show top 10 most frequent GO terms


 GO:0005886    206
 GO:0007186    110
 GO:0007200     74
 GO:0007204     72
 GO:0007187     56
              ... 
GO:0045453       1
GO:0045178       1
GO:0044298       1
 GO:0006468      1
GO:0097746       1
Length: 1994, dtype: int64


In [97]:
# Filter GO terms with at least 3 positives (i.e., sum >= 3)
valid_go_terms = go_term_counts[go_term_counts >= 40].index

# Keep only those columns + the ID
filtered_df = final_df[["ID"] + valid_go_terms.tolist()]


In [98]:
filtered_df

,ID,GO:0005886,GO:0007186,GO:0007200,GO:0007204,GO:0007187,GO:0016020,GO:0004930,GO:0030425,GO:0007268,GO:0007189,GO:0045202,GO:0005886,GO:0007218,GO:0006954
0,O00222,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,O14842,0,0,1,1,0,0,1,0,0,0,0,1,0,0
2,O15303,1,0,0,0,0,0,1,0,0,0,1,0,0,0
3,O43603,1,0,1,1,0,1,0,0,0,1,0,0,1,0
4,O43613,0,0,0,0,0,0,0,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Q9Y271,1,0,0,1,0,0,0,0,0,0,0,0,1,0
261,Q9Y2T5,0,1,0,0,0,0,1,0,0,0,0,1,0,0
262,Q9Y2T6,0,1,1,0,0,0,1,0,0,0,0,1,0,0
263,Q9Y5N1,1,0,0,0,1,0,0,0,1,0,1,0,0,0


In [99]:
# Step 3: Merge ESM embeddings (esm_df) with labels
final_df = pd.merge(esm_df, filtered_df, on="ID")



In [100]:
final_df

,ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,GO:0005886,GO:0007186,GO:0007200,GO:0007204,GO:0007187,GO:0016020,GO:0004930,GO:0030425,GO:0007268,GO:0007189,GO:0045202,GO:0005886,GO:0007218,GO:0006954
0,O00222,0.053351,-0.016727,0.095342,0.019120,-0.040266,0.029943,0.034857,-0.065425,-0.119622,0.084034,0.189269,0.082737,0.141250,0.083907,0.145495,-0.026589,0.085848,-0.114556,0.073753,-0.046051,0.071367,-0.194329,0.029051,0.110599,-0.066128,-0.048791,0.040854,-0.109102,-0.006453,0.053301,0.052703,0.066938,0.062259,0.117598,0.050600,0.075783,-0.069683,0.057038,-0.199899,...,-0.052940,-0.028588,0.067712,-0.151646,0.138573,-0.085640,0.211012,-0.112005,0.072453,0.073242,0.322436,-0.018508,0.116388,0.033675,0.100799,0.002633,-0.038261,-0.152742,-0.250213,-0.019936,-0.046279,-0.066123,-0.053912,-0.319778,0.154516,0.027740,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,O14842,0.033164,0.003283,0.070440,0.017247,-0.000506,-0.001080,0.072019,-0.019934,-0.002518,0.146560,0.066969,0.110360,0.088639,-0.009717,0.126314,-0.007059,0.008345,-0.118922,0.071927,0.052728,0.018556,-0.144658,0.047951,0.026455,-0.116518,0.011171,-0.039262,0.039070,-0.002789,0.017824,-0.033686,0.097200,0.072442,0.116532,0.035961,0.006328,-0.072425,-0.004690,-0.164266,...,-0.022438,0.038202,-0.020310,-0.167656,0.200614,-0.087063,0.172152,-0.180581,0.106033,0.170829,0.308940,-0.058183,0.157688,0.033978,0.051201,-0.035282,-0.050757,-0.043355,-0.268326,0.007200,0.033871,-0.038196,-0.078597,-0.243790,0.139195,0.009131,0,0,1,1,0,0,1,0,0,0,0,1,0,0
2,O15303,0.046058,-0.003895,0.107735,-0.043677,-0.036064,0.051198,0.008938,-0.091502,-0.082031,0.126929,0.140179,0.112133,0.089415,0.036302,0.153926,-0.002192,0.046648,-0.160509,0.042648,-0.029195,0.105228,-0.163157,0.040122,0.057356,-0.106678,-0.013544,0.002152,-0.029846,0.002000,0.028005,0.011248,0.067748,0.056806,0.100574,0.035591,0.032103,-0.006247,0.020242,-0.211365,...,-0.060548,-0.028468,0.063255,-0.167214,0.180808,-0.083491,0.179052,-0.182711,0.063008,0.127183,0.300812,-0.040056,0.113649,0.009707,0.042111,0.007012,-0.083423,-0.134998,-0.274291,0.017472,-0.019652,-0.099833,-0.059169,-0.263769,0.147156,0.046976,1,0,0,0,0,0,1,0,0,0,1,0,0,0
3,O43603,0.026092,0.021502,0.108253,-0.051013,-0.084989,-0.003950,0.036050,-0.097860,-0.047290,0.132933,0.098016,0.133074,0.073828,-0.029753,0.143076,-0.010819,0.014871,-0.172833,0.052151,0.006506,0.010378,-0.170636,0.052968,0.030470,-0.076116,0.003909,-0.032292,0.006818,-0.017088,0.018353,0.009050,0.073158,0.089807,0.110140,0.042149,-0.002068,-0.012955,0.004581,-0.144520,...,-0.037716,0.037388,0.046592,-0.119775,0.191066,-0.087563,0.193291,-0.208645,0.135936,0.130928,0.311979,-0.072137,0.159069,0.023428,0.072661,-0.019724,-0.095362,-0.115929,-0.210134,0.040711,0.048382,-0.081693,-0.079056,-0.241364,0.149850,-0.005024,1,0,1,1,0,1,0,0,0,1,0,0,1,0
4,O43613,0.011840,0.010907,0.097771,-0.025915,-0.035944,0.000257,0.034860,-0.074876,-0.053891,0.112991,0.124297,0.092068,0.090999,0.009306,0.146772,-0.000049,0.018793,-0.142564,0.059508,0.014894,0.006454,-0.169821,0.044034,0.080941,-0.078092,-0.022126,0.022288,-0.020584,-0.030051,0.037274,0.000176,0.052825,0.076747,0.110095,0.037259,0.025773,-0.048181,-0.007014,-0.140095,...,0.005408,0.029117,0.065564,-0.146518,0.178514,-0.082176,0.205130,-0.168356,0.146198,0.110225,0.307986,-0.044313,0.175695,-0.007807,0.074895,0.012051,-0.064174,-0.119595,-0.222651,0.037250,0.032173,-0.075750,-0.087266,-0.271265,0.127608,0.014359,0,0,0,0,0,0,0,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Q9Y

In [101]:
# Step 4: Save final_df to CSV
final_df.to_csv("esm_with_multihot_labels.csv", index=False)
print("Saved: esm_with_multihot_labels.csv")

Saved: esm_with_multihot_labels.csv


## Splitting Column in X(Sequence) and Y(labels)

In [102]:

all_columns = final_df.columns

# GO columns start from index 1281 onwards
#go_columns = all_columns[1281:]

# Now define X and Y
#X = final_df.drop(columns=["ID"] + list(go_columns))  # Features (1280 ESM cols)
#Y = final_df[list(go_columns)]

                    # Labels (GO terms)
all_columns

Index(['ID', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       ' GO:0007187', ' GO:0016020', ' GO:0004930', ' GO:0030425',
       ' GO:0007268', 'GO:0007189', ' GO:0045202', 'GO:0005886', ' GO:0007218',
       ' GO:0006954'],
      dtype='object', length=1295)

In [103]:
# Recalculate GO columns
go_columns = [col for col in final_df.columns if col.startswith("GO:")]
columns = all_columns[1281:]
X = final_df.drop(columns=["ID"] + list(columns))
# Extract labels
Y = final_df[columns]

In [104]:
Y

,GO:0005886,GO:0007186,GO:0007200,GO:0007204,GO:0007187,GO:0016020,GO:0004930,GO:0030425,GO:0007268,GO:0007189,GO:0045202,GO:0005886,GO:0007218,GO:0006954
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,1,1,0,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,1,0,0,0,1,0,0,0
3,1,0,1,1,0,1,0,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,1,0,0,1,0,0,0,0,0,0,0,0,1,0
261,0,1,0,0,0,0,1,0,0,0,0,1,0,0
262,0,1,1,0,0,0,1,0,0,0,0,1,0,0
263,1,0,0,0,1,0,0,0,1,0,1,0,0,0


In [105]:
X.to_csv("X_esm.csv", index=False)
Y.to_csv("Y_go.csv", index=False)

In [106]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("✅ Training samples:", len(X_train))
print("✅ Test samples:", len(X_test))





✅ Training samples: 212
✅ Test samples: 53


In [107]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert DataFrames to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [108]:
from MLP_Model import build_mlp_model
from train_MLP import train_model


In [109]:
# Define sizes
input_size = 1280      # ESM embedding size
hidden_size = 512      # You can adjust this
output_size = Y_train.shape[1]


  # Number of GO term labels


In [110]:
output_size

14

In [111]:
# Build the model
model = build_mlp_model(input_size, hidden_size, output_size)

# Check summary
print(model)

Sequential(
  (0): Linear(in_features=1280, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=14, bias=True)
  (4): Sigmoid()
)


In [112]:
print(Y_train.shape[1])

14


In [113]:
train_model(model, train_loader, val_loader, epochs=20, lr=0.001, threshold=0.3)


Epoch 1/20, Loss: 0.5839, Validation Macro F1 Score: 0.1055
Epoch 2/20, Loss: 0.5204, Validation Macro F1 Score: 0.1055
Epoch 3/20, Loss: 0.5036, Validation Macro F1 Score: 0.1632
Epoch 4/20, Loss: 0.5010, Validation Macro F1 Score: 0.1632
Epoch 5/20, Loss: 0.5026, Validation Macro F1 Score: 0.1259
Epoch 6/20, Loss: 0.4915, Validation Macro F1 Score: 0.1246
Epoch 7/20, Loss: 0.4907, Validation Macro F1 Score: 0.1341
Epoch 8/20, Loss: 0.4881, Validation Macro F1 Score: 0.1578
Epoch 9/20, Loss: 0.4869, Validation Macro F1 Score: 0.1353
Epoch 10/20, Loss: 0.4823, Validation Macro F1 Score: 0.1293
Epoch 11/20, Loss: 0.4816, Validation Macro F1 Score: 0.1624
Epoch 12/20, Loss: 0.4741, Validation Macro F1 Score: 0.1721
Epoch 13/20, Loss: 0.4735, Validation Macro F1 Score: 0.1879
Epoch 14/20, Loss: 0.4666, Validation Macro F1 Score: 0.1888
Epoch 15/20, Loss: 0.4625, Validation Macro F1 Score: 0.2654
Epoch 16/20, Loss: 0.4546, Validation Macro F1 Score: 0.3014
Epoch 17/20, Loss: 0.4540, Valida